# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os
from FACET.Facet import Facet
os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

eegDataSet = "NiazyFMRI.edf"
#eegDataSet = "C:\\Users\\janik\\Downloads\\FMRIB_Data.set\\eegNiazy.edf"

f = Facet()


## Second Step import EEG Data

In [ ]:
f.import_EEG(eegDataSet, rel_trig_pos=-0.01)
f.plot_EEG()

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:
f.find_triggers_with_events(r'\b1\b') # Using Niazys data
#f.cut()
#f.find_triggers_with_events(r'\b1\b',idx=1) # Using Niazys data

#f.find_triggers(r'.*TR.*') # Using Fischmeisters data

## Fourth step apply AAS

In [ ]:
f.apply_MNE_AAS()

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )



## Sixth step Post-Processing

In [ ]:
f.downsample()
f.plot_EEG()
f.lowpass(h_freq=50)

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG()
f.export_EEG("NiazyFMRI_pyFacet_cleaned.edf")

## Eighth Evaluate Results

In [ ]:
eeg = f.get_eeg()
f.add_to_evaluate(eeg)
f.import_EEG('NiazyFMRI_matlab_cleaned.edf')
f.lowpass(h_freq=50)
f.add_to_evaluate(f.get_eeg(), start_time=eeg["time_triggers_start"], end_time=eeg["time_triggers_end"])
f.evaluate(plot=True,measures=["SNR"])

